# Strojenie modelu BERT-base na danych *cyberbullying*

## Biblioteki

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, AutoTokenizer
import torch
from transformers import BertForSequenceClassification, BertForTokenClassification, Trainer, TrainingArguments
import re
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader

## Załadowanie danych

In [2]:
def clean_text(text):
    text = re.sub(r'@anonymized_account', '', text)
    text = re.sub(r'RT', '', text)
    return text

### Teksty

In [3]:
df_sentence = pd.read_excel('cyberbullying_train_complete_sentences.xlsx', sheet_name='Superanotator')
df_sentence = df_sentence[['text', 'LABEL']][:600]

In [4]:
df_sentence['text'] = df_sentence['text'].apply(clean_text)
df_sentence['LABEL'] = df_sentence['LABEL'].astype(int)

df_sentence.head()

,text,LABEL
0,Dla mnie faworytem do tytułu będzie Cracovia. ...,0
1,Brawo ty Daria kibic ma być na dobre i złe,0
2,"Super, polski premier składa kwiaty na groba...",1
3,Musi. Innej drogi nie mamy.,0
4,"Odrzut natychmiastowy, kwaśna mina, mam problem",0


In [5]:
texts_train, texts_temp, text_labels_train, text_labels_temp = train_test_split(
    df_sentence['text'], df_sentence['LABEL'], test_size=0.2, random_state=42
)
texts_val, texts_test, text_labels_val, text_labels_test = train_test_split(
    texts_temp, text_labels_temp, test_size=0.5, random_state=42
)

### Tokeny

In [67]:
df_noun = pd.read_excel('cyberbullying_train_words_noun.xlsx')
df_noun = df_noun[['nouns', 'SUPERanotator']][:600]
df_noun.columns = ['noun', 'LABEL']

# df_noun['LABEL'] = df_noun['LABEL'].replace({0: 1, 1: 2, 2: 3, 3: 4})

df_noun.head()

,noun,LABEL
0,LUDZIE,0.0
1,RESTAURACJA,1.0
2,pomoc,2.0
3,świat,3.0
4,kurtkę,1.0


In [68]:
sorted_labels = sorted(set(df_noun['LABEL'])) 
label2id = dict(zip(sorted_labels, range(0, len(sorted_labels))))
id2label = dict(zip(range(0, len(sorted_labels)), sorted_labels))

tokenizer_noun = AutoTokenizer.from_pretrained('bert-base-uncased')

In [69]:
tokenized_noun = []
labels = []

for noun, label in zip(df_noun['noun'], df_noun['LABEL']):
    # Tokenizacja tekstu
    tokenized_input = tokenizer_noun(noun, add_special_tokens=False)
    # Konwersja etykiet do identyfikatorów
    label_ids = [label2id[label]] * len(tokenized_input['input_ids'])
    
    tokenized_noun.append(noun)
    labels.append(label_ids)

noun_train, noun_temp, labels_train, labels_temp = train_test_split(
    tokenized_noun, labels, test_size=0.2, random_state=42
)
noun_val, noun_test, labels_val, labels_test = train_test_split(
    noun_temp, labels_temp, test_size=0.5, random_state=42
)

## Strojenie modelu dla zadania klasyfikacji tekstu

In [9]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [10]:
def compute_metrics_sentence(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    
    # Wypisanie rzeczywistych i przewidywanych etykiet
    print("Prawdziwe etykiety:", labels)
    print("Przewidywane etykiety:", preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [11]:
tokenizer_sentence = AutoTokenizer.from_pretrained('bert-base-uncased')

text_train_dataset = TextClassificationDataset(texts_train, text_labels_train, tokenizer_sentence)
text_val_dataset = TextClassificationDataset(texts_val, text_labels_val, tokenizer_sentence)
text_test_dataset = TextClassificationDataset(texts_test, text_labels_test, tokenizer_sentence)

model_sentence = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
training_args = TrainingArguments(
    output_dir='./results_sentence',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

trainer_sentence = Trainer(
    model=model_sentence,
    args=training_args,
    train_dataset=text_train_dataset,
    eval_dataset=text_val_dataset,
    compute_metrics=compute_metrics_sentence
)

d:\Python\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer_sentence.train()

  0%|          | 0/120 [00:00<?, ?it/s]

KeyError: 304

In [ ]:
eval_results_sentence = trainer_sentence.evaluate()
print("Wyniki ewaluacji:", eval_results_sentence)

In [14]:
noun_model_path = 'sentence_model-weights/'
model_sentence.save_pretrained(noun_model_path)

## Strojenie modelu dla zadanie klasyfikacji tokenów

In [56]:
class TokenClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        
        # Tworzymy tensor etykiet o długości max_length
        label_ids = torch.tensor(label + [0] * (self.max_length - len(label)), dtype=torch.long)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label_ids
        }

In [57]:
def compute_metrics(pred):
    predictions, labels = pred
    predictions = predictions.argmax(axis=-1)

    # Spłaszczamy predykcje i etykiety, usuwając padding (wartości 0)
    true_labels = []
    pred_labels = []
    for true, pred in zip(labels, predictions):
        true_labels.extend(true[true != -100].tolist())
        pred_labels.extend(pred[true != -100].tolist())

    accuracy = accuracy_score(true_labels, pred_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='weighted', zero_division=1)
    
    print("True Labels:", true_labels)
    print("Predicted Labels:", pred_labels)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [70]:
train_dataset = TokenClassificationDataset(noun_train, labels_train, tokenizer_noun)
val_dataset = TokenClassificationDataset(noun_val, labels_val, tokenizer_noun)
test_dataset = TokenClassificationDataset(noun_test, labels_test, tokenizer_noun)

In [72]:
model_noun = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(label2id))


training_args = TrainingArguments(
    output_dir='./results_noun',
    num_train_epochs=15,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch", 
    save_strategy="epoch"         
)

trainer_noun = Trainer(
    model=model_noun,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [73]:
print("Starting to train...")
trainer_noun.train()

Starting to train...


  0%|          | 0/3600 [00:00<?, ?it/s]

{'loss': 0.5384, 'grad_norm': 1.1920520067214966, 'learning_rate': 4.986111111111111e-05, 'epoch': 0.04}
{'loss': 0.0411, 'grad_norm': 0.4705689251422882, 'learning_rate': 4.972222222222223e-05, 'epoch': 0.08}
{'loss': 0.0441, 'grad_norm': 0.8288436532020569, 'learning_rate': 4.958333333333334e-05, 'epoch': 0.12}
{'loss': 0.0252, 'grad_norm': 0.31944742798805237, 'learning_rate': 4.9444444444444446e-05, 'epoch': 0.17}
{'loss': 0.0369, 'grad_norm': 0.39272540807724, 'learning_rate': 4.930555555555556e-05, 'epoch': 0.21}
{'loss': 0.0352, 'grad_norm': 0.36144089698791504, 'learning_rate': 4.9166666666666665e-05, 'epoch': 0.25}
{'loss': 0.0251, 'grad_norm': 0.42043256759643555, 'learning_rate': 4.902777777777778e-05, 'epoch': 0.29}
{'loss': 0.0448, 'grad_norm': 0.4744585156440735, 'learning_rate': 4.888888888888889e-05, 'epoch': 0.33}
{'loss': 0.034, 'grad_norm': 0.16211245954036713, 'learning_rate': 4.875e-05, 'epoch': 0.38}
{'loss': 0.0309, 'grad_norm': 0.2519978880882263, 'learning_rate

  0%|          | 0/30 [00:00<?, ?it/s]

True Labels: [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

...

  0%|          | 0/30 [00:00<?, ?it/s]

True Labels: [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

  0%|          | 0/30 [00:00<?, ?it/s]

True Labels: [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

TrainOutput(global_step=3600, training_loss=0.009323740438493487, metrics={'train_runtime': 1409.9342, 'train_samples_per_second': 5.107, 'train_steps_per_second': 2.553, 'total_flos': 470342666649600.0, 'train_loss': 0.009323740438493487, 'epoch': 15.0})

In [74]:
eval_results = trainer_noun.evaluate()
print("Wyniki ewaluacji:", eval_results)

  0%|          | 0/30 [00:00<?, ?it/s]

True Labels: [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [75]:
# Zapisanie modelu
noun_model_path = 'noun_model-weights/'
model_noun.save_pretrained(noun_model_path)

## Ocena jakości predyckji

### Teksty

In [76]:
predictions = trainer_sentence.predict(text_test_dataset)
metrics_sentence = predictions.metrics

predicted_labels_id2label_sentence = predictions.predictions.argmax(axis=1)
true_labels_id2label_sentence = predictions.label_ids

# Wypisanie wyników i porównanie przewidywań z rzeczywistymi etykietami
print("Wyniki:", metrics_sentence, "\n")

for i in range(len(predicted_labels_id2label_sentence)):
    print(f"Sample {i + 1}:")
    print("Predicted:", predicted_labels_id2label_sentence[i])
    print("True:", true_labels_id2label_sentence[i])
    print("=" * 100)

NameError: name 'trainer_sentence' is not defined

### Tokeny

In [77]:
predictions_noun, labels, metrics = trainer_noun.predict(test_dataset)

predicted_labels = predictions_noun.argmax(-1).tolist()  # Przewidywane etykiety
true_labels = labels.tolist()                       # Rzeczywiste etykiety


# Konwersja ID na etykiety tekstowe
predicted_labels_text = [[id2label[id] for id in pred] for pred in predicted_labels]
true_labels_text = [[id2label[id] for id in true] for true in true_labels]

print("Wyniki:", metrics, "\n")

for i in range(len(predicted_labels_text)):
    print(f"Sample {i + 1}:")
    print("Predicted labels:", predicted_labels_text[i])
    print("True labels:     ", true_labels_text[i])
    print("=" * 100)

  0%|          | 0/30 [00:00<?, ?it/s]

True Labels: [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Wizualizacja przestrzeni wektorowej

### Tokeny

In [78]:
nouns = test_dataset.texts
test_encodings = [tokenizer_noun(
    text,
    add_special_tokens=True,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)['input_ids'].flatten() for text in nouns]

X = np.array(list(test_encodings))

#### PCA

In [79]:
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(X)

df_noun_pca = pd.DataFrame(reduced_vectors, columns=['PCA1', 'PCA2'])
df_noun_pca['Predicted Label']  = [label[0] for label in predicted_labels_text]
df_noun_pca['Noun'] = noun_test
df_noun_pca['True label'] = [label[0] for label in true_labels_text]

fig = px.scatter(df_noun_pca, x="PCA1", y="PCA2", color="Predicted Label", hover_data={'Noun': True, 'True label': True}, title="Wizualizacja przestrzeni wektorowej prób testowych",
                labels={"PC1": "Pierwsza składowa główna", "PC2": "Druga składowa główna"})
fig.show()

#### TSNE

In [80]:
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)

X_embedded = TSNE(n_components=2, random_state=42).fit_transform(X_standardized)

df_tsne = pd.DataFrame(X_embedded, columns=['x', 'y'])
df_tsne['Predicted_label'] = [label[0] for label in predicted_labels_text]
df_tsne['Noun'] = noun_test
df_tsne['True_label'] = [label[0] for label in true_labels_text]

fig = px.scatter(df_tsne, x='x', y='y', color='Predicted_label', hover_data={'Noun': True, 'True_label': True}, title="Wizualizacja przestrzeni wektorowej prób testowych")
fig.show()